# Car Rental Optimization with cuOpt Python API

This notebook demonstrates how to solve a car rental optimization problem using the cuOpt Python API. The problem involves determining the optimal fleet size and distribution of vehicles to maximize weekly profit.

## Problem Description

A car rental company operates in multiple locations. They need to determine:
- How many cars to have in their fleet
- Where to position cars each day
- How many cars to transfer between locations

The goal is to maximize weekly profit while:
- Meeting rental demand at each location
- Considering transfer costs between locations
- Respecting operational constraints

### Model Assumptions

**Important**: This model makes the following simplifying assumptions:
1. **Same-day rentals**: All cars rented on a given day are returned to the same location by end of day
2. **Daily optimization horizon**: Decisions are made on a daily basis over a one-week period
3. **Known demand**: Rental demand at each location is known in advance
4. **Homogeneous fleet**: All cars are identical (no vehicle classes)
5. **Overnight transfers**: Car transfers between locations happen overnight and are available the next day

This formulation is well-suited for:
- Airport shuttle services
- City car-sharing programs
- Daily rental operations
- Short-term rental businesses

**Note**: For multi-day rentals where cars don't return the same day, the model would need additional variables to track rental duration and expected return dates.

This problem is inspired by the classic car rental optimization example and is formulated as a Mixed Integer Linear Program (MILP).

**Reference**: Adapted from [Gurobi's Vehicle Rental Optimization Model](https://www.gurobi.com/jupyter_models/vehicle-rental-optimization/)


## Environment Setup

First, let's check if we have a GPU available and install necessary dependencies.


In [ ]:
import subprocess
import html
from IPython.display import display, HTML

def check_gpu():
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, timeout=5)
        result.check_returncode()
        lines = result.stdout.splitlines()
        gpu_info = lines[2] if len(lines) > 2 else "GPU detected"
        gpu_info_escaped = html.escape(gpu_info)
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{gpu_info_escaped}</pre>
        </div>
        """))
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired, FileNotFoundError, IndexError) as e:
        display(HTML("""
        <div style="border:2px solid red;padding:15px;border-radius:10px;background:#ffeeee;">
            <h3>⚠️ GPU not detected!</h3>
            <p>This notebook requires a <b>GPU runtime</b>.</p>
            
            <h4>If running in Google Colab:</h4>
            <ol>
              <li>Click on <b>Runtime → Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then click <b>Save</b> and <b>Runtime → Restart runtime</b>.</li>
            </ol>
            
            <h4>If running in Docker:</h4>
            <ol>
              <li>Ensure you have <b>NVIDIA Docker runtime</b> installed (<code>nvidia-docker2</code>)</li>
              <li>Run container with GPU support: <code>docker run --gpus all ...</code></li>
              <li>Or use: <code>docker run --runtime=nvidia ...</code> for older Docker versions</li>
              <li>Verify GPU access: <code>docker run --gpus all nvidia/cuda:12.0.0-base-ubuntu22.04 nvidia-smi</code></li>
            </ol>
            
            <p><b>Additional resources:</b></p>
            <ul>
              <li><a href="https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html" target="_blank">NVIDIA Container Toolkit Installation Guide</a></li>
            </ul>
        </div>
        """))
        return False

check_gpu()


In [ ]:
# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed
#!pip uninstall -y cuda-python cuda-bindings cuda-core
#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu12 nvidia-nvjitlink-cu12 rapids-logger==0.1.19
#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu13 nvidia-nvjitlink-cu13 rapids-logger==0.1.19


In [ ]:
!pip install matplotlib seaborn

## Import Required Libraries


In [ ]:
import numpy as np
import pandas as pd
from cuopt.linear_programming.problem import Problem, VType, sense, LinearExpression
from cuopt.linear_programming.solver_settings import SolverSettings
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## Problem Data Setup

Define the locations, time periods, rental revenue, transfer costs, and demand data.


In [ ]:
# Define locations and time horizon
locations = ['Airport', 'Downtown', 'Suburb_A', 'Suburb_B']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

n_locations = len(locations)
n_days = len(days)

print(f"Optimizing car rental for {n_locations} locations over {n_days} days")
print(f"Locations: {', '.join(locations)}")


In [ ]:
# Revenue per car rental at each location (in $)
rental_revenue = {
    'Airport': 120,
    'Downtown': 100,
    'Suburb_A': 80,
    'Suburb_B': 75
}

# Weekly operational cost per car (leasing, insurance, maintenance, etc.)
# This replaces the one-time purchase cost to make the weekly optimization meaningful
# Amortized from ~$25,000 purchase over 2 years = ~$240/week
weekly_car_cost = 240

print(f"Weekly operational cost per car: ${weekly_car_cost}")

# Transfer cost matrix (cost to move one car from location i to location j)
# Rows: source location, Columns: destination location
transfer_cost_matrix = np.array([
    [0,   50,  40,  45],  # from Airport
    [50,  0,   30,  35],  # from Downtown
    [40,  30,  0,   25],  # from Suburb_A
    [45,  35,  25,  0 ]   # from Suburb_B
])

print("\nTransfer Cost Matrix ($):")
transfer_df = pd.DataFrame(transfer_cost_matrix, index=locations, columns=locations)
print(transfer_df)


In [ ]:
# Demand: number of cars requested at each location on each day
# Rows: days, Columns: locations
demand = np.array([
    [25, 15, 10, 8],   # Monday
    [20, 18, 12, 10],  # Tuesday
    [22, 20, 11, 9],   # Wednesday
    [24, 16, 13, 11],  # Thursday
    [30, 22, 15, 12],  # Friday
    [35, 25, 18, 14],  # Saturday
    [28, 20, 14, 10]   # Sunday
])

# Return rates: percentage of rented cars that return to the same location
# (1 - return_rate) cars need to be transferred back
return_rate = np.array([0.7, 0.8, 0.85, 0.85])  # for each location

print("\nDemand by Day and Location:")
demand_df = pd.DataFrame(demand, index=days, columns=locations)
print(demand_df)


In [ ]:
# Visualize demand patterns
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Heatmap of demand
sns.heatmap(demand, annot=True, fmt='d', cmap='YlOrRd', 
            xticklabels=locations, yticklabels=days, ax=axes[0])
axes[0].set_title('Demand Heatmap (Cars per Day)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Location', fontsize=12)
axes[0].set_ylabel('Day', fontsize=12)

# Line plot of demand over time
for i, loc in enumerate(locations):
    axes[1].plot(days, demand[:, i], marker='o', label=loc, linewidth=2)
axes[1].set_title('Demand Trends Over the Week', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Day', fontsize=12)
axes[1].set_ylabel('Number of Cars Demanded', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## Problem Formulation

### Decision Variables
- `fleet_size`: Total number of cars in the fleet (integer)
- `available[d][l]`: Number of cars available at location `l` on day `d` (continuous)
- `rented[d][l]`: Number of cars rented at location `l` on day `d` (continuous)
- `transfer[d][i][j]`: Number of cars transferred from location `i` to location `j` on day `d` (continuous)

### Objective Function
Maximize: Total Revenue - Weekly Operational Costs - Transfer Costs

### Constraints
1. **Demand constraint**: Cars rented cannot exceed demand
2. **Availability constraint**: Cars rented cannot exceed available cars
3. **Flow conservation**: Cars available at a location = previous day's remaining + transfers in - transfers out
4. **Fleet size constraint**: Total cars in the system cannot exceed fleet size


In [ ]:
# Create the optimization problem
problem = Problem("car_rental_optimization")

# Decision variable: Fleet size (total number of cars to purchase)
fleet_size = problem.addVariable(name="fleet_size", vtype=VType.INTEGER, lb=0.0, ub=500.0)

print(f"Created fleet size variable")


In [ ]:
# Decision variables: Cars available at each location on each day
available = {}
for d in range(n_days):
    available[d] = {}
    for l in range(n_locations):
        var = problem.addVariable(
            name=f"available_day{d}_loc{l}",
            vtype=VType.CONTINUOUS,
            lb=0.0,
            ub=500.0
        )
        available[d][l] = var

print(f"Created {n_days * n_locations} availability variables")


In [ ]:
# Decision variables: Cars rented at each location on each day
rented = {}
for d in range(n_days):
    rented[d] = {}
    for l in range(n_locations):
        var = problem.addVariable(
            name=f"rented_day{d}_loc{l}",
            vtype=VType.CONTINUOUS,
            lb=0.0,
            ub=float(demand[d][l])  # Cannot rent more than demand
        )
        rented[d][l] = var

print(f"Created {n_days * n_locations} rental variables")


In [ ]:
# Decision variables: Cars transferred between locations each day
transfer = {}
for d in range(n_days):
    transfer[d] = {}
    for i in range(n_locations):
        transfer[d][i] = {}
        for j in range(n_locations):
            if i != j:  # No self-transfers
                var = problem.addVariable(
                    name=f"transfer_day{d}_from{i}_to{j}",
                    vtype=VType.CONTINUOUS,
                    lb=0.0,
                    ub=100.0
                )
                transfer[d][i][j] = var

n_transfer_vars = n_days * n_locations * (n_locations - 1)
print(f"Created {n_transfer_vars} transfer variables")


## Set Objective Function

Maximize total profit = rental revenue - weekly operational costs - transfer costs


In [ ]:
# Build objective function
objective = LinearExpression([], [], 0.0)

# Revenue from rentals
for d in range(n_days):
    for l in range(n_locations):
        loc_name = locations[l]
        revenue = float(rental_revenue[loc_name])
        objective += rented[d][l] * revenue

# Subtract weekly operational costs
objective += fleet_size * float(-weekly_car_cost)

# Subtract transfer costs
for d in range(n_days):
    for i in range(n_locations):
        for j in range(n_locations):
            if i != j:
                cost = float(transfer_cost_matrix[i][j])
                objective += transfer[d][i][j] * (-cost)

# Set objective to maximize profit
problem.setObjective(objective, sense.MAXIMIZE)

print("Objective function set: Maximize (Revenue - Weekly Operational Costs - Transfer Costs)")


## Add Constraints


In [ ]:
# Constraint 1: Rented cars cannot exceed available cars
for d in range(n_days):
    for l in range(n_locations):
        problem.addConstraint(
            rented[d][l] <= available[d][l],
            name=f"rental_limit_day{d}_loc{l}"
        )

print(f"Added {n_days * n_locations} rental limit constraints")


In [ ]:
# Constraint 2: Flow conservation
# For day 0: available[0][l] = initial allocation
# For day d > 0: available[d][l] = available[d-1][l] - rented[d-1][l] + transfers_in - transfers_out + returns

# Initial distribution constraint (day 0)
initial_allocation_expr = LinearExpression([], [], 0.0)
for l in range(n_locations):
    initial_allocation_expr += available[0][l]
    
problem.addConstraint(
    initial_allocation_expr <= fleet_size,
    name="initial_fleet_allocation"
)

print("Added initial fleet allocation constraint")


In [ ]:
# Flow conservation for subsequent days
# Assumption: Cars are rented and returned within the same day
# So rented cars are still counted in the flow, they're just temporarily unavailable
for d in range(1, n_days):
    for l in range(n_locations):
        # Cars available today = cars from yesterday + transfers in - transfers out
        # (Rented cars come back the same day, so they don't leave the location's inventory)
        flow_expr = LinearExpression([], [], 0.0)
        
        # Start with available cars from previous day
        flow_expr += available[d-1][l]
        
        # Add incoming transfers from previous day
        for i in range(n_locations):
            if i != l:
                flow_expr += transfer[d-1][i][l]
        
        # Subtract outgoing transfers from previous day
        for j in range(n_locations):
            if j != l:
                flow_expr += transfer[d-1][l][j] * (-1.0)
        
        # This should equal available cars today
        problem.addConstraint(
            available[d][l] == flow_expr,
            name=f"flow_conservation_day{d}_loc{l}"
        )

print(f"Added {(n_days - 1) * n_locations} flow conservation constraints")


In [ ]:
# Constraint 3: Total cars in system cannot exceed fleet size (for each day)
# Since rented cars are returned the same day and stay at the location,
# we only count available cars (which includes cars that will be rented that day)
for d in range(n_days):
    total_cars_expr = LinearExpression([], [], 0.0)
    for l in range(n_locations):
        total_cars_expr += available[d][l]
    
    problem.addConstraint(
        total_cars_expr <= fleet_size,
        name=f"fleet_size_limit_day{d}"
    )

print(f"Added {n_days} fleet size limit constraints")


## Solve the Problem


In [ ]:
# Configure solver settings
solver_settings = SolverSettings()
solver_settings.time_limit = 60.0  # 60 second time limit

print("Starting optimization...")
print(f"Problem size: {problem.NumVariables} variables, {problem.NumConstraints} constraints")
print()

# Solve the problem
start_time = time.time()
problem.solve(solver_settings)
solve_time = time.time() - start_time

print(f"\nOptimization completed in {solve_time:.3f} seconds")
print(f"Solution status: {problem.Status.name}")


## Analyze Results


In [ ]:
# Extract solution values
optimal_fleet_size = fleet_size.getValue()
optimal_profit = problem.ObjValue

print("=" * 70)
print(" " * 20 + "OPTIMIZATION RESULTS")
print("=" * 70)
print(f"\nOptimal Fleet Size: {int(optimal_fleet_size)} cars")
print(f"Total Profit: ${optimal_profit:,.2f}")
print(f"Average Daily Profit: ${optimal_profit/n_days:,.2f}")
print()


In [ ]:
# Extract availability data
availability_matrix = np.zeros((n_days, n_locations))
rental_matrix = np.zeros((n_days, n_locations))

for d in range(n_days):
    for l in range(n_locations):
        availability_matrix[d][l] = available[d][l].getValue()
        rental_matrix[d][l] = rented[d][l].getValue()

# Create DataFrames for better visualization
availability_df = pd.DataFrame(availability_matrix, index=days, columns=locations)
rental_df = pd.DataFrame(rental_matrix, index=days, columns=locations)

print("\nAvailable Cars by Day and Location:")
print(availability_df.round(1))
print("\n" + "="*70 + "\n")

print("Rented Cars by Day and Location:")
print(rental_df.round(1))


In [ ]:
# Calculate utilization rates
utilization_matrix = np.zeros((n_days, n_locations))
for d in range(n_days):
    for l in range(n_locations):
        avail = availability_matrix[d][l]
        if avail > 0:
            utilization_matrix[d][l] = (rental_matrix[d][l] / avail) * 100

utilization_df = pd.DataFrame(utilization_matrix, index=days, columns=locations)

print("\n" + "="*70 + "\n")
print("Utilization Rate (%) by Day and Location:")
print(utilization_df.round(1))


In [ ]:
# Calculate demand satisfaction
demand_satisfaction = np.zeros((n_days, n_locations))
for d in range(n_days):
    for l in range(n_locations):
        if demand[d][l] > 0:
            demand_satisfaction[d][l] = (rental_matrix[d][l] / demand[d][l]) * 100

demand_satisfaction_df = pd.DataFrame(demand_satisfaction, index=days, columns=locations)

print("\n" + "="*70 + "\n")
print("Demand Satisfaction Rate (%) by Day and Location:")
print(demand_satisfaction_df.round(1))
print(f"\nOverall Demand Satisfaction: {demand_satisfaction.mean():.1f}%")


In [ ]:
# Analyze transfer patterns
total_transfers = 0
transfer_costs_total = 0

print("\n" + "="*70)
print("TRANSFER OPERATIONS")
print("="*70 + "\n")

for d in range(n_days):
    day_transfers = []
    for i in range(n_locations):
        for j in range(n_locations):
            if i != j:
                transfer_count = transfer[d][i][j].getValue()
                if transfer_count > 0.5:  # Only show significant transfers
                    total_transfers += transfer_count
                    transfer_costs_total += transfer_count * transfer_cost_matrix[i][j]
                    day_transfers.append({
                        'From': locations[i],
                        'To': locations[j],
                        'Cars': round(transfer_count, 1),
                        'Cost': f"${transfer_count * transfer_cost_matrix[i][j]:.2f}"
                    })
    
    if day_transfers:
        print(f"{days[d]}:")
        transfer_day_df = pd.DataFrame(day_transfers)
        print(transfer_day_df.to_string(index=False))
        print()

print(f"\nTotal Transfers: {total_transfers:.1f} cars")
print(f"Total Transfer Costs: ${transfer_costs_total:,.2f}")


## Visualize Results


In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Available Cars Heatmap
sns.heatmap(availability_matrix, annot=True, fmt='.1f', cmap='Blues',
            xticklabels=locations, yticklabels=days, ax=axes[0, 0])
axes[0, 0].set_title('Available Cars by Day and Location', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Location', fontsize=12)
axes[0, 0].set_ylabel('Day', fontsize=12)

# 2. Rented Cars Heatmap
sns.heatmap(rental_matrix, annot=True, fmt='.1f', cmap='Greens',
            xticklabels=locations, yticklabels=days, ax=axes[0, 1])
axes[0, 1].set_title('Rented Cars by Day and Location', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Location', fontsize=12)
axes[0, 1].set_ylabel('Day', fontsize=12)

# 3. Utilization Rate Heatmap
sns.heatmap(utilization_matrix, annot=True, fmt='.1f', cmap='YlOrRd',
            xticklabels=locations, yticklabels=days, ax=axes[1, 0])
axes[1, 0].set_title('Utilization Rate (%) by Day and Location', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Location', fontsize=12)
axes[1, 0].set_ylabel('Day', fontsize=12)

# 4. Demand Satisfaction Heatmap
sns.heatmap(demand_satisfaction, annot=True, fmt='.1f', cmap='RdYlGn',
            xticklabels=locations, yticklabels=days, ax=axes[1, 1], vmin=0, vmax=100)
axes[1, 1].set_title('Demand Satisfaction Rate (%) by Day and Location', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Location', fontsize=12)
axes[1, 1].set_ylabel('Day', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# Revenue breakdown by location
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Total rentals by location
total_rentals_by_location = rental_matrix.sum(axis=0)
axes[0].bar(locations, total_rentals_by_location, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
axes[0].set_title('Total Rentals by Location (Entire Week)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Location', fontsize=12)
axes[0].set_ylabel('Number of Rentals', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, v in enumerate(total_rentals_by_location):
    axes[0].text(i, v + 1, f'{v:.0f}', ha='center', va='bottom', fontweight='bold')

# Revenue by location
revenue_by_location = [total_rentals_by_location[i] * rental_revenue[locations[i]] 
                       for i in range(n_locations)]
axes[1].bar(locations, revenue_by_location, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
axes[1].set_title('Total Revenue by Location (Entire Week)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Location', fontsize=12)
axes[1].set_ylabel('Revenue ($)', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, v in enumerate(revenue_by_location):
    axes[1].text(i, v + 100, f'${v:,.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()


## Summary and Business Insights


In [ ]:
# Calculate key metrics
total_revenue = sum(revenue_by_location)
total_operational_cost = optimal_fleet_size * weekly_car_cost
avg_utilization = utilization_matrix.mean()
avg_demand_satisfaction = demand_satisfaction.mean()
total_demand = demand.sum()
total_rentals = rental_matrix.sum()

print("\n" + "="*70)
print(" " * 20 + "BUSINESS INSIGHTS")
print("="*70 + "\n")

print(f"📊 Financial Performance:")
print(f"   • Total Revenue: ${total_revenue:,.2f}")
print(f"   • Weekly Operational Cost: ${total_operational_cost:,.2f}")
print(f"   • Transfer Costs: ${transfer_costs_total:,.2f}")
print(f"   • Net Profit: ${optimal_profit:,.2f}")
if total_operational_cost > 0:
    print(f"   • Profit Margin: {(optimal_profit / total_revenue) * 100:.1f}%")
else:
    print(f"   • Profit Margin: N/A")
print()

print(f"🚗 Fleet Performance:")
print(f"   • Optimal Fleet Size: {int(optimal_fleet_size)} cars")
print(f"   • Average Utilization Rate: {avg_utilization:.1f}%")
print(f"   • Total Rentals: {int(total_rentals)} out of {int(total_demand)} demanded")
print(f"   • Overall Demand Satisfaction: {avg_demand_satisfaction:.1f}%")
print()

print(f"🔄 Operational Efficiency:")
print(f"   • Total Car Transfers: {int(total_transfers)} cars moved")
print(f"   • Average Daily Transfers: {total_transfers/n_days:.1f} cars/day")
print(f"   • Transfer Cost per Car: ${transfer_costs_total/total_transfers if total_transfers > 0 else 0:.2f}")
print()

# Find best and worst performing locations
location_revenue = {locations[i]: revenue_by_location[i] for i in range(n_locations)}
best_location = max(location_revenue, key=location_revenue.get)
worst_location = min(location_revenue, key=location_revenue.get)

print(f"📍 Location Performance:")
print(f"   • Best Performing: {best_location} (${location_revenue[best_location]:,.2f})")
print(f"   • Needs Improvement: {worst_location} (${location_revenue[worst_location]:,.2f})")
print()

print(f"⚡ Optimization Performance:")
print(f"   • Solve Time: {solve_time:.3f} seconds")
print(f"   • Problem Size: {problem.NumVariables} variables, {problem.NumConstraints} constraints")
print(f"   • GPU Acceleration: Enabled ✓")
print("\n" + "="*70 + "\n")


## Conclusion

This notebook demonstrated how to use cuOpt linear programming API to solve a car rental optimization problem inspired by the Gurobi modeling example.

### What We Achieved:
1. **Optimal fleet size** to maximize profit while meeting demand
2. **Daily allocation** of vehicles across locations
3. **Transfer recommendations** to rebalance inventory efficiently
4. **Performance metrics** for business decision-making

### Key Advantages of cuOpt:
- **GPU Acceleration**: Significantly faster than CPU-based solvers
- **Scalability**: Handles problems with thousands of variables and constraints
- **Integration**: Easy to integrate into existing Python workflows
- **Real-time**: Fast enough for operational decision-making

### Current Model Assumptions:
This model assumes **same-day rentals** where all cars return to their origin location by end of day. This simplification works well for:
- Car-sharing services (Zipcar, Car2Go)
- Airport rental shuttles
- Daily urban rentals

However, traditional car rentals often involve **multi-day bookings** and **one-way rentals** (rent at airport, return downtown).

### Possible Extensions:

**High Priority - Multi-day Rentals:**
- Track rental duration with variables `rent_start[d][l]` and `rent_duration[d][l]`
- Add constraints for car availability during rental period
- Model one-way rentals allowing cars rented at location i to be returned at location j
- Include reservation system with time windows for pre-booked rentals

**Additional Enhancements:**
- Add seasonal demand variations and holiday effects
- Include different vehicle types and classes (economy, SUV, luxury)
- Model maintenance schedules and vehicle downtime
- Add stochastic elements for uncertain demand
- Optimize over longer time horizons (monthly, quarterly)
- Include customer service levels and overbooking strategies



SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.